# **Architecture et modélisation des données**

Dans ce chapitre, nous allons explorer les concepts fondamentaux de l'architecture des données et de la modélisation des données. Nous aborderons les différents types de modèles de données, les principes de conception d'une architecture de données efficace, ainsi que les meilleures pratiques pour la gestion et l'organisation des données au sein d'une organisation.

## **L'architecture des données**

### Le Data Warehouse : L'entrepôt structuré

Le Data Warehouse (Entrepôt de Données), est né dans les années 1990 pour répondre à un besoin : centraliser les données d'entreprise pour faciliter l'analyse et la prise de décision. Imaginez une entreprise avec des dizaines de systèmes différents : un logiciel de comptabilité, un CRM pour les clients, un système de gestion des stocks, des outils RH. Chaque système fonctionne en silo, avec ses propres données. Le Data Warehouse propose de rassembler toutes ces données dans un lieu unique et organisé.

La métaphore de la bibliothèque est particulièrement appropriée pour comprendre le Data Warehouse. Dans une bibliothèque bien organisée, chaque livre a sa place précise : les romans sont classés par auteur, les documentaires par sujet, les magazines par date de parution. De la même manière, dans un Data Warehouse, chaque donnée est structurée selon un schéma prédéfini. On parle de "schema-on-write", c'est-à-dire qu'on définit la structure avant d'y stocker quoi que ce soit.

Les caractéristiques principales d'un Data Warehouse sont 4. Premièrement, il est historisé : on ne supprime jamais les données, on les accumule dans le temps pour pouvoir analyser les tendances. Deuxièmement, il est non volatile : une fois qu'une donnée est écrite, elle n'est plus modifiée, ce qui garantit la fiabilité des analyses. Troisièmement, il est intégré : les données proviennent de sources multiples mais sont harmonisées selon des règles communes. Enfin, il est orienté sujet : on organise les données par thème métier plutôt que par application source.

Concrètement, prenons l'exemple d'une banque. Son Data Warehouse contiendrait toutes les informations sur les comptes clients, l'historique de toutes les transactions depuis des années, les détails de tous les prêts accordés, et les informations sur tous les placements. Ces données sont structurées dans des tables avec des relations clairement définies. Si un analyste veut savoir combien de clients ont ouvert un compte épargne au premier trimestre 2024, il peut interroger directement le Data Warehouse avec une requête simple et obtenir une réponse rapide et fiable.

Cependant, le Data Warehouse présente aussi des limites. Sa rigidité est à la fois sa force et sa faiblesse. Il excelle avec les données structurées (tableaux, nombres, textes formatés), mais il ne peut pas gérer facilement des données non structurées comme des images, des vidéos, ou des fichiers JSON complexes. De plus, ajouter un nouveau type de données nécessite souvent de revoir le schéma, ce qui peut être long et coûteux.

#### Les 2 écoles de pensée : Inmon versus Kimball

L'histoire du Data Warehouse est marquée par deux visions différentes, portées par deux pionniers : Bill Inmon et Ralph Kimball. Comprendre cette opposition est fondamental car elle structure encore aujourd'hui notre façon de penser l'architecture des données.

##### **L'approche Inmon : Top-Down**

Bill Inmon, souvent appelé "le père du Data Warehouse", prône une approche centralisée et méthodique. Sa philosophie peut se résumer ainsi : avant de pouvoir analyser quoi que ce soit, il faut d'abord construire un Data Warehouse d'entreprise complet et parfaitement normalisé. Ce n'est qu'ensuite qu'on en dérive des Data Marts spécialisés par département.

La normalisation est au cœur de cette approche. Normaliser signifie organiser les données de manière à éliminer toute redondance. Prenons un exemple concret avec un site e-commerce. Sans normalisation, chaque fois qu'un client passe commande, on répéterait toutes ses informations : son nom, son email, son adresse. Si Marie Dubois passe 10 commandes, on aurait 10 fois les mêmes informations. Avec la normalisation à la Inmon, on crée une table "Clients" où Marie apparaît une seule fois avec toutes ses informations, et une table "Commandes" où chaque commande pointe simplement vers Marie par un identifiant.

L'intégrité des données est maximale : si l'email de Marie change, on modifie une seule ligne dans la table Clients, et automatiquement toutes ses commandes voient le changement. On a une seule source de vérité pour toute l'entreprise : le département ventes et le département marketing utilisent exactement la même définition de "client". L'architecture est robuste et évolutive : elle peut tenir vingt ans sans nécessiter de refonte majeure.

Mais ces avantages ont un prix. Le temps de mise en œuvre est considérable : il faut compter entre 12 et 24 mois avant de voir les premiers résultats. Pourquoi si long ? Parce qu'il faut d'abord analyser toute l'entreprise, comprendre tous les processus métier, harmoniser toutes les définitions, avant même de commencer à construire quoi que ce soit. Le coût est également élevé, nécessitant une équipe importante d'architectes, de DBA (Database Administrators), et d'analystes métier. Enfin, les utilisateurs métier doivent attendre longtemps avant de pouvoir utiliser le système, ce qui peut générer de la frustration et même conduire à l'abandon du projet.

##### **L'approche Kimball : Bottom-Up**

Ralph Kimball, le pragmatique, propose une vision radicalement différente. Sa philosophie : pourquoi attendre 2 ans pour avoir un système complet alors qu'on pourrait commencer à créer de la valeur dès maintenant ? Construisons d'abord des Data Marts dimensionnels par domaine métier, et intégrons-les progressivement.

Avec Kimball, on commence par identifier le processus métier le plus important, par exemple les ventes, et on construit rapidement un Data Mart spécifiquement pour ce domaine. Ce Data Mart utilise ce qu'on appelle un "modèle en étoile", que nous détaillerons dans la section sur la modélisation. L'idée est d'accepter une certaine redondance des données pour optimiser la performance des analyses. Une fois le Data Mart Ventes en production, on passe au Data Mart Marketing, puis au Data Mart RH, et ainsi de suite.

Le concept clé chez Kimball est le "Bus Dimensionnel". C'est une métaphore empruntée aux transports en commun : différentes lignes de bus (les Data Marts) partagent les mêmes arrêts (les dimensions). Concrètement, si le Data Mart Ventes et le Data Mart Marketing utilisent tous deux une dimension "Client" avec exactement la même structure et les mêmes définitions, ils pourront communiquer et permettre des analyses croisées. C'est ce qu'on appelle des dimensions "conformées".

Les avantages de l'approche Kimball sont particulièrement attractifs pour les organisations qui veulent des résultats rapides. Le retour sur investissement est rapide : un premier Data Mart peut être en production en quatre à huit semaines. Les requêtes sont simples et performantes : au lieu de devoir croiser six ou sept tables comme dans l'approche Inmon, on en croise généralement deux ou trois. Le modèle est intuitif pour les utilisateurs métier : la structure en étoile est facile à comprendre, même pour des non-techniciens. Enfin, l'approche est itérative et agile : on peut commencer petit, prouver la valeur, puis étendre progressivement.

Mais Kimball a aussi ses inconvénients. La redondance assumée coûte en espace de stockage : les informations sur Marie Dubois peuvent être répétées dans plusieurs Data Marts. Il y a un risque de créer des silos si les Data Marts ne respectent pas le principe des dimensions conformées : le département ventes et le département marketing pourraient finir par avoir des définitions légèrement différentes de "client", rendant les analyses croisées impossibles. Enfin, la maintenance devient complexe à grande échelle : gérer vingt Data Marts différents demande une coordination importante.

##### **Comparaison et choix**

Aujourd'hui, en 2025, le débat Inmon versus Kimball n'est plus aussi tranché qu'avant. La plupart des organisations adoptent une approche hybride, empruntant le meilleur des deux mondes. Les grandes entreprises avec des exigences réglementaires strictes (banques, assurances) tendent encore vers Inmon pour son intégrité et sa robustesse. Les startups, PME et organisations agiles préfèrent généralement Kimball pour sa rapidité et sa flexibilité. Et comme nous le verrons plus loin, le Modern Data Stack avec son architecture en couches (**Bronze**, **Silver**, **Gold**) permet de combiner élégamment les deux approches.